In [1]:
from dotenv import load_dotenv
import os
import pandas as pd

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
load_dotenv()
open_api_key = os.environ['OPENAI_API_KEY']

In [3]:
df = pd.read_csv("data/train/fraudTrain.csv", index_col=0).sample(1000)

In [4]:
df.sample(3)

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
856480,2019-12-16 14:37:56,342351256941125,fraud_Schiller Ltd,personal_care,61.68,Rebecca,Obrien,F,5619 Mendoza Inlet,Juliette,...,33.1194,-83.8235,3343,Theatre manager,1990-06-08,0390ca27042b9661c0a978f4ef7afc8d,1355668676,32.614363,-83.284819,0
120443,2019-03-09 04:33:25,6595970453799027,"fraud_Stehr, Jewess and Schimmel",misc_net,5.89,Gina,Gomez,F,72030 Shelton Crossing Apt. 742,Stirling,...,40.6774,-74.4968,2307,Science writer,1969-09-11,4a85ff9678d9b01f9e84c16fbea63112,1331267605,40.523987,-73.799561,0
593973,2019-09-09 00:27:14,4890424426862856940,fraud_Cummings LLC,gas_transport,70.56,Jeremy,Whitney,M,0371 Aimee Neck Suite 856,Sacramento,...,38.5662,-121.3286,757530,Magazine features editor,1979-04-30,d62804ad948f0a84b475f9b55303eddf,1347150434,38.101123,-121.736792,0


In [5]:
from langchain.agents import create_pandas_dataframe_agent
from langchain.chat_models import ChatOpenAI

agent = create_pandas_dataframe_agent(
    ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
    df,
    agent_type="openai-functions",
    agent_executor_kwargs={"handle_parsing_errors": True},
    verbose=True
)

In [6]:
template = """
            You are an expert python developer and datascientist working with a pandas dataframe in Python. 
            Your goal is to create a good exploratory data analysis of the dataframe.
            The name of the dataframe is `df`.
            You should use the tools below to answer the question posed of you:
            
            python_repl_ast: A Python shell. Use this to execute python commands. Input should be a valid python command. 
            When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.
            
            Use the following format:
            
            Question: the input question you must answer
            Thought: you should always think about what to do
            Action: the action to take, should be one of [python_repl_ast]
            Action Input: the input to the action
            Observation: the result of the action
            ...(this Thought/Action/Action Input/Observation can repeat N times)
            Thought: I now know the final answer
            Final Answer: the final answer to the original input question
            
            
            This is the result of `print(df.head())`:
            {df_head}
            
            Begin!
            
            Question: {input}
            {agent_scratchpad}
"""

In [7]:
agent.run("What is the shape of the dataframe?")



> Entering new AgentExecutor chain...
Thought: To find the shape of the dataframe, I can use the `shape` attribute of the dataframe.

Action: python_repl_ast
Action Input: df.shape
Observation: (1000, 22)
Thought:The shape of the dataframe is (1000, 22).
Final Answer: (1000, 22)

> Finished chain.


'(1000, 22)'

In [9]:
agent.run("Perform initial explorations to get a high-level understanding of our dataset")



> Entering new AgentExecutor chain...
Thought: I should check the shape of the dataframe to see how many rows and columns it has. I should also check the data types of each column to understand the types of data we have.

Action: python_repl_ast
Action Input: df.shape
Observation: (1000, 22)
Thought:The dataframe has 1000 rows and 22 columns.
Thought: I should check the data types of each column to understand the types of data we have.

Action: python_repl_ast
Action Input: df.dtypes
Observation: trans_date_trans_time     object
cc_num                     int64
merchant                  object
category                  object
amt                      float64
first                     object
last                      object
gender                    object
street                    object
city                      object
state                     object
zip                        int64
lat                      float64
long                     float64
city_pop                   int64
j

'The dataframe has 1000 rows and 22 columns, with various data types and no missing values.'

In [10]:
agent.run("""Of the transactions from personal care category that are significantly higher than the average of transactions made within this category, how many of them are fraud transactions?""")



> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe to only include transactions from the personal care category. Then, I need to calculate the average transaction amount for this category. Finally, I need to filter the dataframe again to only include transactions with amounts higher than the average and check how many of them are fraud transactions.

Action: python_repl_ast
Action Input: df[df['category'] == 'personal_care']['amt'].mean()
Observation: 46.641975308641975
Thought:The average transaction amount for the personal care category is 46.641975308641975.

Action: python_repl_ast
Action Input: df[(df['category'] == 'personal_care') & (df['amt'] > 46.641975308641975)]['is_fraud'].sum()
Observation: 0
Thought:There are no fraud transactions among the transactions from the personal care category that are significantly higher than the average transaction amount for this category.

Final Answer: 0

> Finished chain.


'0'

In [11]:
agent.run("Between fraud and non-fraud transactions, which one has a higher average of transaction amount?")



> Entering new AgentExecutor chain...
Thought: I need to calculate the average transaction amount for fraud and non-fraud transactions separately and compare them.

Action: Use pandas groupby() function to group the dataframe by the 'is_fraud' column and calculate the average of the 'amt' column.

Action Input: df.groupby('is_fraud')['amt'].mean()

Observation: Use pandas groupby() function to group the dataframe by the 'is_fraud' column and calculate the average of the 'amt' column. is not a valid tool, try another one.
Thought:I need to use the pandas groupby() function to group the dataframe by the 'is_fraud' column and calculate the average of the 'amt' column. I should use the python_repl_ast tool to execute this command.

Action: python_repl_ast

Action Input: df.groupby('is_fraud')['amt'].mean()

Observation: is_fraud
0     69.000834
1    500.668000
Name: amt, dtype: float64
Thought:The average transaction amount for non-fraud transactions is 69.000834, while the average trans

'Fraud transactions have a higher average transaction amount.'